In [ ]:
import pyalex
from pyalex import Sources
from pyalex import Works
import itertools
import pandas as pd

In [ ]:
# Give email to use polite pool, otherwise leave blank
pyalex.config.email = input()

## Get sources

In [ ]:
field = "techlaw"
journals = pd.read_csv("journals/" + field + ".csv")

In [ ]:
len(journals)

In [ ]:
# Get OA sources by journal issn
# Iterate journals in smaller chunks, otherwise seems not to return all results
chunks = [list(journals['issn_l'])[i:i+10] for i in range(0,len(journals),10)]

sources = []
for c in chunks:
    query = "|".join( c )
    results = Sources().filter(issn=query).get()
    sources += results

len(sources)

#### Check and drop duplicates

In [ ]:
# Turn to sources dataframe for duplicate removal
sources = pd.DataFrame(sources)

In [ ]:
sources[sources.duplicated(subset='issn_l', keep=False)]

In [ ]:
sources.drop_duplicates(subset='issn_l', keep="last", inplace=True)
len(sources)

#### Check missing journals

In [ ]:
# Turn back to dicts
sources = sources.to_dict('records')

In [ ]:
issn_list = [s['issn_l'] for s in sources]
missing = journals[journals['issn_l'].apply( lambda issn: issn not in issn_list )]
missing

In [ ]:
# Try to get missing journals by title
missing_titles = list(missing['journal'])

for title in missing_titles:
    results = Sources().filter(display_name={"search":title}).get()
    sources += results
    if results:
        missing = missing[missing['journal'] != title]
        
# Write missing journals to file
missing.to_csv("data/" + field + "_missing.csv", index=False)
missing

In [ ]:
# Should be True
len(sources) + len(missing) == len(journals)

print(len(sources), "sources in total for", field)

## Get works

In [ ]:
s_ids_names = {s['id'].replace('https://openalex.org/','') : s['display_name'] for s in sources}

In [ ]:
import time
works = []
for s_id in s_ids_names.keys():
    print("Getting source:", s_ids_names[s_id])
    paginator = Works().filter(primary_location={"source" : {"id" : s_id}}).paginate(per_page=200)
    works += list(itertools.chain.from_iterable(paginator))
    time.sleep(30)

In [ ]:
df = pd.DataFrame(works)
df['abstract'] = [w['abstract'] for w in works]

In [ ]:
df.to_csv("data/" + field + ".csv", index=False)
df.to_json("data/" + field + ".json")